 # Classification des avis sur des vêtements de femmes vendus dans le e-commerce

- Est ce que les avis que l'on a des vêtements sont représentatifs de la note qui est attribuée ?

Idées :
- visualisation de données : quels types de vêtements ont les notes les plus élevées ?
- nb d'avis donné selon l'âge des clients

In [143]:
import pandas as pd
import spacy


## I. Import des données

Dans un premier temps, nous allons importer nos données. Notre base de données contient des informations sur des avis de vêtements femmes vendus sur internet. Ces données sont issus d'un processus de webscrapping.

In [144]:
# Import des données
data = pd.read_csv("Womens Clothing E-Commerce Reviews.csv", sep = ",")

# Renomage première colonne pour pouvoir l'utiliser comme id par la suite
data = data.rename(columns = {"Unnamed: 0" : "id"})

# Affichage des 5 premières lignes
data.head()

,id,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [145]:
data.shape

(23486, 11)

Notre jeu de données contient 23 486 avis et 11 colonnes. 

## II. Pré-traitement des données

Pour avoir des données plus propres, nous allons effectuer divers pré-traitements.

Pour commencer, nous allons supprimer les lignes où nous avons des valeurs manquantes. 

In [146]:
# Suppression des valeurs manquantes
data = data.dropna()
data.shape

(19662, 11)

Suite à cette manipulation, nous supprimons environ 4 000 lignes pour pouvoir avoir des données complètes pour poursuivre notre analyse.

### Récupération des données pour notre étude

### Traitement de la casse

- suppression des valeurs manquantes => pas d'avis : inutile
- suppression des caractères spéciaux 
- suppression des majuscules
- suppression des mots vides
- lemmatisation 
- affiche du nombre de mots par étiquette grammaticale
- extraction des mots (groupes de mots) les plus fréquents

- wordcloud

Dans un premier temps, nous allons récupérer les avis.

In [147]:
avis = data["Review Text"]
avis

2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
5        I love tracy reese dresses, but this one is no...
6        I aded this in my basket at hte last mintue to...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 19662, dtype: object

In [148]:
type(avis)

pandas.core.series.Series

Nous allons récupérer seulement la partie textuelle de l'avis, cela nous permet de ne avoir un objet Pandas.Series.

In [149]:
from pprint import pprint

liste_avis = data["Review Text"].values.tolist()
pprint(liste_avis[:10])

['I had such high hopes for this dress and really wanted it to work for me. i '
 'initially ordered the petite small (my usual size) but i found this to be '
 'outrageously small. so small in fact that i could not zip it up! i reordered '
 'it in petite medium, which was just ok. overall, the top half was '
 'comfortable and fit nicely, but the bottom half had a very tight under layer '
 'and several somewhat cheap (net) over layers. imo, a major design flaw was '
 'the net over layer sewn directly into the zipper - it c',
 "I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time "
 'i wear it, i get nothing but great compliments!',
 'This shirt is very flattering to all due to the adjustable front tie. it is '
 'the perfect length to wear with leggings and it is sleeveless so it pairs '
 'well with any cardigan. love this shirt!!!',
 'I love tracy reese dresses, but this one is not for the very petite. i am '
 'just under 5 feet tall and usually wear a 0p in this 

Grâce à cette manipulation, chaque avis est élément d'une liste d'avis. 

Ensuite, nous allons découper les avis en liste de mots et les mettre en minuscules pour pouvoir les analyser plus facilement.

In [150]:
# Découpage des avis en mots

liste_avis_clean = []

for avis in liste_avis : 
    avis = str(avis)
    avis_clean = avis.split()
    avis_clean = avis.lower()
    liste_avis_clean.append(avis_clean)

liste_avis_clean[:10]

['i had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
 "i love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments!",
 'this shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!',
 'i love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress was very pretty out of

Puis, nous allons tockeniser notre texte. 

In [151]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
liste_avis_clean = [tokenizer.tokenize(str(avis)) for avis in liste_avis_clean]
liste_avis_clean[:10]

[['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
  'outrageously',
  'small',
  'so',
  'small',
  'in',
  'fact',
  'that',
  'i',
  'could',
  'not',
  'zip',
  'it',
  'up',
  'i',
  'reordered',
  'it',
  'in',
  'petite',
  'medium',
  'which',
  'was',
  'just',
  'ok',
  'overall',
  'the',
  'top',
  'half',
  'was',
  'comfortable',
  'and',
  'fit',
  'nicely',
  'but',
  'the',
  'bottom',
  'half',
  'had',
  'a',
  'very',
  'tight',
  'under',
  'layer',
  'and',
  'several',
  'somewhat',
  'cheap',
  'net',
  'over',
  'layers',
  'imo',
  'a',
  'major',
  'design',
  'flaw',
  'was',
  'the',
  'net',
  'over',
  'layer',
  'sewn',
  'directly',
  'into',
  'the',
  'zipper',
  'it',
  'c'],
 ['i',
  'love',
  'love',
 

Ensuite, on supprime la ponctuation.

In [152]:
import string
punct = string.punctuation

# Pour chaque token dans chaque avis, si le token n'est pas dans la liste des ponctuations, on le garde
liste_avis_clean = [[token for token in avis if token not in punct] for avis in liste_avis_clean]
liste_avis_clean[:10]

[['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
  'outrageously',
  'small',
  'so',
  'small',
  'in',
  'fact',
  'that',
  'i',
  'could',
  'not',
  'zip',
  'it',
  'up',
  'i',
  'reordered',
  'it',
  'in',
  'petite',
  'medium',
  'which',
  'was',
  'just',
  'ok',
  'overall',
  'the',
  'top',
  'half',
  'was',
  'comfortable',
  'and',
  'fit',
  'nicely',
  'but',
  'the',
  'bottom',
  'half',
  'had',
  'a',
  'very',
  'tight',
  'under',
  'layer',
  'and',
  'several',
  'somewhat',
  'cheap',
  'net',
  'over',
  'layers',
  'imo',
  'a',
  'major',
  'design',
  'flaw',
  'was',
  'the',
  'net',
  'over',
  'layer',
  'sewn',
  'directly',
  'into',
  'the',
  'zipper',
  'it',
  'c'],
 ['i',
  'love',
  'love',
 

## III. Classification

### Traitement et séparation des données

Dans cette partie, nous allons chercher à classifier les avis en fonction de leur note. 

Nous allons utiliser la colonne "Rating" comme étiquettes et "Review Text" comme valeurs. 




#### Sélection des informations dans notre dataframe

Nous allons sélectionner les trois colonnes qui vont nous servir pour la classification dans un objectif d'optimiser les temps de calculs et de ne pas avoir d'informations superflus.

In [153]:
# On récupère la colonne id, Rating et Review Text
data = data[["id", "Rating", "Review Text"]]
data.head()

,id,Rating,Review Text
2,2,3,I had such high hopes for this dress and reall...
3,3,5,"I love, love, love this jumpsuit. it's fun, fl..."
4,4,5,This shirt is very flattering to all due to th...
5,5,2,"I love tracy reese dresses, but this one is no..."
6,6,5,I aded this in my basket at hte last mintue to...


#### Analyse de la colonne "Rating"

In [154]:
# Analyse de la colonne "Rating"
data["Rating"].value_counts()

Rating
5    10858
4     4289
3     2464
2     1360
1      691
Name: count, dtype: int64

Nous avons ici des notes allant de 1 à 5. Nous allons diviser ces valeurs en 3 catégories : 
- -1 pour les notes allant de 1 à 2
- 0 pour les notes égales à 3 
- 1 pour les plus élevées (4 et 5)

#### Analyse de la colonne "Review Text"

Notre colonne correspondant aux valeurs est "Review Text". \
Cette colonne contient tous les avis laissés par les internautes sur les différents vêtements.

#### Changement des étiquettes

Pour réaliser notre classification, nous allons donc modifier les étiquettes comme précisé ci-dessus. 

In [155]:
def map_label_to_numeric(label):
    return 1 if label == 5 else 0 if label == 3 or label == 4 else -1

In [156]:
def get_labels(data):
    labels = data[["id","Rating"]]
    labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)
    labels.set_index('id', inplace=True)
    
    # ajouter les labels dans data selon l'id
    data['score_avis'] = labels

    # data['score_avis'] = labels
    return data

In [157]:
data = get_labels(data)
data.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7908\29256686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)


,id,Rating,Review Text,score_avis
2,2,3,I had such high hopes for this dress and reall...,0
3,3,5,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,5,This shirt is very flattering to all due to th...,1
5,5,2,"I love tracy reese dresses, but this one is no...",-1
6,6,5,I aded this in my basket at hte last mintue to...,1


In [158]:
# On analyse la nouvelle colonne "score_avis"
data["score_avis"].value_counts()

score_avis
 1    10858
 0     6753
-1     2051
Name: count, dtype: int64

Grâce à cette manipulation, nous pouvons observer que les avis ayant la note de 5 sont majoritaires dans notre jeu de données puisque cela correspond à la note de 1. Les avis compris entre 1 et 2 ont une proportion plus faible (-1). 

A présent, nous n'avons plus besoin de la colonne Rating, nous pouvons donc la supprimer du dataframe.

In [159]:
data = data[["id", "Review Text", "score_avis"]]
data.head()

,id,Review Text,score_avis
2,2,I had such high hopes for this dress and reall...,0
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,This shirt is very flattering to all due to th...,1
5,5,"I love tracy reese dresses, but this one is no...",-1
6,6,I aded this in my basket at hte last mintue to...,1


#### Division de notre dataframe

Pour réaliser notre classification, nous avons besoin de séparer notre jeu de données en un jeu d'apprentissage, de validation et de test.

Note :
Les données en apprentissage automatique sont généralement séparées en trois jeux :
+ **entraînement** : données destinées à l'apprentissage du modèle ;
+ **validation** : données destinées à une évaluation intermédiaire du modèle pour permettre l'ajustement de ses hyperparamètres. Une fois les hyperparamètres du modèle arrêtés, on peut le ré-entraîner sur l'ensemble des données (entraînement + validation) avant de le tester sur le jeu de test ;
+ **test** : données destinées EXCLUSIVEMENT à l'évaluation FINALE (à réaliser une fois uniquement !) du modèle choisi finalement. Elles ne doivent sous aucune forme servir à la conception du modèle. Il est donc interdit aussi bien de les examiner que d'évaluer le modèle en cours de développement sur ce jeu de données.

Pour créer l'ensemble de validation, nous allons effectuer la manipulation à la fin du pré-traitement réalisé lors de la classification. 

In [160]:
# Fonction pour diviser de notre jeu de données en 2 : train et test
def split_data(data, train_ratio):
    data_train = data.sample(frac = train_ratio)
    data_test = data.drop(data_train.index)
    return data_train, data_test

# Diviser notre jeu de données en 2 : train et test
data_train, data_test = split_data(data, 0.6)

In [161]:
data_train.shape, data_test.shape

((11797, 3), (7865, 3))

Dans notre cas :
+ entraînement (appelé *Train*) contenant 11797 observations ;
+ validation (appelé *Validation*) contenant 5243 observations ;
+ test (appelé *Test*), contenant 2622 observations, soit environ 22% de la taille du jeu d'entraînement.

In [162]:
data_train.head()

,id,Review Text,score_avis
10556,10556,I love this top! it's so hard for me to find c...,1
3282,3282,I absolutely fell in love with the design of t...,0
14016,14016,This dress is beautiful--lovely colors and des...,0
19217,19217,Cute top. definitely sheer so need an undershi...,0
14821,14821,I was just at the store and purchased this cut...,1


In [163]:
data_test.head()

,id,Review Text,score_avis
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,This shirt is very flattering to all due to th...,1
5,5,"I love tracy reese dresses, but this one is no...",-1
6,6,I aded this in my basket at hte last mintue to...,1
7,7,"I ordered this in carbon for store pick up, an...",0


### Exploration des données

#### Distribution des classes

Il est important de connaître la répartition des classes dans les données d'entraînement pour pouvoir procéder à notre classification.

In [164]:
# Analyse de la colonne "score_avis" de notre jeu de données d'entrainement
print(data_train["score_avis"].value_counts())

# Calcul des proportions de chaque classe dans notre jeu de données d'entrainement
data_train["score_avis"].value_counts()/len(data_train)

score_avis
 1    6497
 0    4075
-1    1225
Name: count, dtype: int64


score_avis
 1    0.550733
 0    0.345427
-1    0.103840
Name: count, dtype: float64

Nous pouvons observer que les classes de scores sont réparties de manière aléatoire dans notre jeu d'apprentissage. Nous pouvons noter plus de 50% d'avis très favorables, correspondant à la note de 5/5. Les avis négatifs sont en minorité dans notre jeu d'entraînement.

#### Exploration du texte

Pour se faire une idée des textes auxquels nous avons affaire, nous allons les afficher pour savoir quels pré-traitements sont nécessaires.

In [165]:
# Affichage des 5 premiers avis
data_train["Review Text"].values[:5]

array(["I love this top! it's so hard for me to find conservative, comfortable and yet not mom-isn swimsuits. this top is amazing. it's comfortable and soft and great quality. i'm excited to use this suit this season!",
       'I absolutely fell in love with the design of this dress! the colors and cut are easy to wear but unique enough to stand out. i was between the medium and the small and ultimately went with the medium. the dress hits right above my knee, fits nicely at my waist, and is well made. the top does seem a bit loose (as pictured on the model) but i prefer a slightly more fitted look. a padded bra would probably have filled out the extra fabric but it was an easy alteration and will allow me to wear the',
       "This dress is beautiful--lovely colors and design--but it's huuuge. i'm usually a s/m in tops and dresses, but i got an xs after trying it on in the store. i'm busty and have hips and still have a bit of room. the length is good--a little above the knee but stil

## Exploration des données

In [253]:
from collections import Counter
import pandas as pd

class_distribution = (pd.DataFrame.from_dict(Counter(data_train.score_avis.values),
                                             orient='index')
                                  .rename(columns={0: 'num_examples'}))
class_distribution.index.name = 'class'
class_distribution

,num_examples
class,
1,6497
0,4075
-1,1225


In [257]:
import numpy as np

class_distribution['perc_examples'] = np.around(class_distribution.num_examples /
                                                np.sum(class_distribution.num_examples),
                                                2)
class_distribution

,num_examples,perc_examples
class,,
1,6497,0.55
0,4075,0.35
-1,1225,0.10


## Représentation des textes

### Sélection de descripteurs : prétraitements textuels

#### Exemple sur un avis

In [166]:
tw = data_train['Review Text'].iloc[100]
tw

"Short of calling it dark gray- which  isn't dark, but that's fine because i saw the picture. it's everything i wanted it to be. i just wish it was chiller here in georgia so i can wear it more often"

Pour effectuer nos tâches de traitements de langage, nous allons transformer le 1er avis en utilisant spacy. En effet, spacy a plusieurs fonctionnalités : 
- permet de tokeniser directement notre texte
- peut lemmatiser les mots
- identifier et classer les entités nommées
- analyser les dépendances syntaxiques entre les mots
- représenter les mots sous forme de vecteurs (embedding)
- etc

In [167]:
# Pour la langue anglaise
!pip install -U spacy
! python -m spacy validate

import spacy
!python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')


⠙ Loading compatibility table...
⠹ Loading compatibility table...
⠸ Loading compatibility table...
⠼ Loading compatibility table...
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.2) =================
ℹ spaCy installation:
c:\Users\ASUS\anaconda3\Lib\site-packages\spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.7.2,<3.8.0   3.7.1   ✔
fr_core_news_sm   >=3.7.0,<3.8.0   3.7.0   ✔

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 682.7 kB/s eta 0:00:19
     --------------------------------------- 0.0/12.8 MB 495.5 kB/s eta 0:00:26
      --------------------------------------- 0.2/12.8 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.4/12.8 MB 2.4 MB/s eta 0:00:06
     -- ------------------------------------- 0.7/12.8 MB 3.1 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/12.8 M

In [168]:
# Transformation du premier avis en objet spacy
avis_nlp = nlp(avis) 
avis_nlp

This dress in a lovely platinum is feminine and fits perfectly, easy to wear and comfy, too! highly recommend!

In [169]:
type(avis_nlp)

spacy.tokens.doc.Doc

On a bien un objet de type spacy.

On affiche chaque token de notre objet spacy :

In [170]:
for token in avis_nlp:
    print(token)

This
dress
in
a
lovely
platinum
is
feminine
and
fits
perfectly
,
easy
to
wear
and
comfy
,
too
!
highly
recommend
!


Ensuite, nous allons pouvoir afficher les lemmes de chaque token. Grâce à cette étape, nous allons pouvoir simplifier les mots pour faciliter notre analyse textuelle.

In [171]:
# Lemmatisation
for token in avis_nlp:
    print(token.lemma_)

this
dress
in
a
lovely
platinum
be
feminine
and
fit
perfectly
,
easy
to
wear
and
comfy
,
too
!
highly
recommend
!


#### Généralisation de la lemmatisation


Pour lemmatiser notre texte, nous allons définir une fonction. Cette étape est indispensable pour récupérer les lemmes des mots de notre texte d'origine. Nous allons simplifier notre texte grâce à cette fonction.

Cette fonction va nous permettre de généraliser par la suite nos manipulations.

In [210]:
def lemmatise_text(text):
    text = nlp(text) # on transforme le texte en objet spacy
    lemmas = [token.lemma_ for token in text] # on récupère les lemmes
    return ' '.join(lemmas) # on retourne les lemmes sous forme de texte

In [211]:
# On teste sur 3 avis 
for avis in liste_avis[:3]:
    print("Avis initial : ", avis) # On affiche l'avis initial
    print("Avis lemmatisé : ", lemmatise_text(avis)) # On applique la fonction à notre avis

Avis initial :  I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c
Avis lemmatisé :  I have such high hope for this dress and really want it to work for I . I initially order the petite small ( my usual size ) but I find this to be outrageously small . so small in fact that I could not zip it up ! I reorder it in petite medium , which be just ok . overall , the top half be comfortable and fit nicely , but the bottom half have a very tight under layer and several somewhat cheap ( net ) over layer . imo , a major design flaw be the net over layer

Nous allons ensuite ajouter une colonne avec la fonction de **lemmatisation** appliquée à nos avis.

In [174]:
data_train['lemmas'] = data_train['Review Text'].apply(lemmatise_text)

In [175]:
data_train.head()

,id,Review Text,score_avis,lemmas
10556,10556,I love this top! it's so hard for me to find c...,1,I love this top ! it be so hard for I to find ...
3282,3282,I absolutely fell in love with the design of t...,0,I absolutely fall in love with the design of t...
14016,14016,This dress is beautiful--lovely colors and des...,0,this dress be beautiful -- lovely color and de...
19217,19217,Cute top. definitely sheer so need an undershi...,0,cute top . definitely sheer so need an undersh...
14821,14821,I was just at the store and purchased this cut...,1,I be just at the store and purchase this cute ...


On effectue la même manipulation sur l'ensemble de test .

In [176]:
data_test['lemmas'] = data_test['Review Text'].apply(lemmatise_text)

In [177]:
data_test.shape

(7865, 4)

In [178]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

##### Racines

Pour réduire les mots à leur forme de base, nous allons utiliser SnowballStemmer sur notre texte. Pour cela, nous allons créer une fonction et l'appliquer à nos avis. 

In [179]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [183]:
def stem_text(text):
    stemmer = SnowballStemmer('english')
    tokenizer = RegexpTokenizer('\w+')
    stems = [stemmer.stem(token) for token in tokenizer.tokenize(text)]
    return ' '.join(stems)

In [184]:
stem_text(avis)

'this dress in a love platinum is feminin and fit perfect easi to wear and comfi too high recommend'

On applique la fonction à notre dataframe.

In [185]:
data_train['stems'] = data_train['Review Text'].apply(stem_text)

In [186]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems
10556,10556,I love this top! it's so hard for me to find c...,1,I love this top ! it be so hard for I to find ...,i love this top it s so hard for me to find co...
3282,3282,I absolutely fell in love with the design of t...,0,I absolutely fall in love with the design of t...,i absolut fell in love with the design of this...
14016,14016,This dress is beautiful--lovely colors and des...,0,this dress be beautiful -- lovely color and de...,this dress is beauti love color and design but...
19217,19217,Cute top. definitely sheer so need an undershi...,0,cute top . definitely sheer so need an undersh...,cute top definit sheer so need an undershirt r...
14821,14821,I was just at the store and purchased this cut...,1,I be just at the store and purchase this cute ...,i was just at the store and purchas this cute ...


On fait de même sur l'ensemble de test.

In [187]:
data_test['stems'] = data_test['Review Text'].apply(stem_text)

In [188]:
data_test.shape

(7865, 5)

In [189]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

##### Étiquettes morphosyntaxiques

Puis, nous allons analyser notre texte et renvoyer chaque mot remplacé par sa catégorie grammaticale pour continuer l'étude des avis.

In [190]:
def replace_words_with_pos_tag(text):
    text = nlp(text)
    return ' '.join([token.pos_ for token in text])

On teste la fonction sur un avis.

In [191]:
replace_words_with_pos_tag(avis)

'DET NOUN ADP DET ADJ NOUN AUX ADJ CCONJ VERB ADV PUNCT ADJ PART VERB CCONJ ADJ PUNCT ADV PUNCT ADV VERB PUNCT'

On applique la fonction à notre ensemble d'entrainement et on ajoute une colonne à notre dataframe.

In [192]:
data_train['pos'] = data_train['Review Text'].apply(replace_words_with_pos_tag)

In [193]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems,pos
10556,10556,I love this top! it's so hard for me to find c...,1,I love this top ! it be so hard for I to find ...,i love this top it s so hard for me to find co...,PRON VERB DET NOUN PUNCT PRON AUX ADV ADJ SCON...
3282,3282,I absolutely fell in love with the design of t...,0,I absolutely fall in love with the design of t...,i absolut fell in love with the design of this...,PRON ADV VERB ADP NOUN ADP DET NOUN ADP DET NO...
14016,14016,This dress is beautiful--lovely colors and des...,0,this dress be beautiful -- lovely color and de...,this dress is beauti love color and design but...,DET NOUN AUX ADJ PUNCT ADJ NOUN CCONJ NOUN PUN...
19217,19217,Cute top. definitely sheer so need an undershi...,0,cute top . definitely sheer so need an undersh...,cute top definit sheer so need an undershirt r...,ADJ NOUN PUNCT ADV ADJ ADV VERB DET NOUN PUNCT...
14821,14821,I was just at the store and purchased this cut...,1,I be just at the store and purchase this cute ...,i was just at the store and purchas this cute ...,PRON AUX ADV ADP DET NOUN CCONJ VERB DET ADJ N...


On effectue la même manipulation sur notre ensemble de test.

In [194]:
data_test['pos'] = data_test['Review Text'].apply(replace_words_with_pos_tag)

In [195]:
data_test.shape

(7865, 6)

In [196]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

#####  Classe d'appartenance des entités nommées

Pour pouvoir effectuer la reconnaissances d'entités nommées sur nos avis, nous allons retourner une version de notre avis ou chaque entité nommée est remplacée par son type d'entité.

In [213]:
def ner(text):

    text = nlp(text) # on transforme le texte en objet spacy
    
    new_text = [] # on crée une liste vide

    for token in text: # pour chaque token dans l'avis

        # print(token.text, token.ent_iob_, token.ent_type_)
        
        if token.ent_iob_ == "O": # si l'entité ne fait pas partie d'une entité nommée
            new_text.append(token.text) # on ajoute le texte du token à la liste
        elif token.ent_iob_ == "B": # si l'entité fait partie d'une entité nommée
            new_text.append(token.ent_type_) # on ajoute le type de l'entité à la liste

        # Si l'entité comprend plusieurs mot on ne répète pas l'étiquette
        else:
            continue
    return ' '.join(new_text) # on retourne les étiquettes sous forme de texte

On applique la fonction sur un avis.

In [214]:
print("Avis initiaux : ", avis)
print("Avec les entités nommées : ", ner(avis))

Avis initiaux :  This shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!
Avec les entités nommées :  This shirt is very flattering to all due to the adjustable front tie . it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan . love this shirt ! ! !


On applique ensuite notre fonction à notre dataframe d'entrainement. 

In [200]:
data_train['entites_nommees'] = data_train['Review Text'].apply(ner)

In [201]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems,pos,entites_nommees
10556,10556,I love this top! it's so hard for me to find c...,1,I love this top ! it be so hard for I to find ...,i love this top it s so hard for me to find co...,PRON VERB DET NOUN PUNCT PRON AUX ADV ADJ SCON...,I love this top ! it 's so hard for me to find...
3282,3282,I absolutely fell in love with the design of t...,0,I absolutely fall in love with the design of t...,i absolut fell in love with the design of this...,PRON ADV VERB ADP NOUN ADP DET NOUN ADP DET NO...,I absolutely fell in love with the design of t...
14016,14016,This dress is beautiful--lovely colors and des...,0,this dress be beautiful -- lovely color and de...,this dress is beauti love color and design but...,DET NOUN AUX ADJ PUNCT ADJ NOUN CCONJ NOUN PUN...,This dress is beautiful -- lovely colors and d...
19217,19217,Cute top. definitely sheer so need an undershi...,0,cute top . definitely sheer so need an undersh...,cute top definit sheer so need an undershirt r...,ADJ NOUN PUNCT ADV ADJ ADV VERB DET NOUN PUNCT...,Cute top . definitely sheer so need an undersh...
14821,14821,I was just at the store and purchased this cut...,1,I be just at the store and purchase this cute ...,i was just at the store and purchas this cute ...,PRON AUX ADV ADP DET NOUN CCONJ VERB DET ADJ N...,I was just at the store and purchased this cut...


Aussi, on applique à notre ensemble de test.

In [215]:
data_test['entites_nommees'] = data_test['Review Text'].apply(ner)

In [216]:
data_test.shape

(7865, 7)

In [217]:
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

###  Calcul des valeurs des descripteurs

Pour procéder aux calculs, nous allons séparer notre jeu de données d'entraînement pour avoir un jeu de données de validation. Les données test nou servirons pour l'évaluation finale des modèles. 

In [218]:
from sklearn.model_selection import train_test_split

In [219]:
X_train, X_valid, y_train, y_valid = train_test_split(data_train['Review Text'],
                                                      data_train['score_avis'],
                                                      train_size=0.75,
                                                      random_state=5)

In [220]:
X_train.shape, X_valid.shape

((8847,), (2950,))

On a donc 8 847 lignes dans notre jeu d'entrainement et 2 950 dans celui de validation.

In [221]:
y_train

2955     0
21614   -1
15031    1
5773     1
19552   -1
        ..
9405     0
617      0
19144   -1
11891    0
2431     1
Name: score_avis, Length: 8847, dtype: int64

Nous pouvons observer que les sorties à prédire correspondent aux trois étiquettes que nous avons défini plus haut.

Pour évaluer notre modèle, nous initialisons les ensembles de test.

In [222]:
# On récupère les avis et les labels du jeu de données de test
X_test, y_test = data_test['Review Text'], data_test['score_avis'] 

### Binaire : présence/absence

In [223]:
from sklearn.feature_extraction.text import CountVectorizer

bin_count = CountVectorizer(binary=True)

In [224]:
bin_count.fit(X_train)
bin_count

CountVectorizer(binary=True)

In [225]:
X_train_vectorized_bin = bin_count.transform(X_train)
X_train_vectorized_bin

<8847x9587 sparse matrix of type '<class 'numpy.int64'>'
	with 386829 stored elements in Compressed Sparse Row format>

In [226]:
X_valid_vectorized_bin = bin_count.transform(X_valid)
X_test_vectorized_bin = bin_count.transform(X_test)

In [227]:
X_valid_vectorized_bin # MEME NOMBRE DE COLONNES QUE X_train_vectorized_bin

<2950x9587 sparse matrix of type '<class 'numpy.int64'>'
	with 128132 stored elements in Compressed Sparse Row format>

###  Numérique discret : décomptes d'occurrence

Nous allons calculer les fréquences d'occurence des termes dans nos avis. 

In [228]:
vect_count = CountVectorizer().fit(X_train)

Nous pouvons examiner le vocabulaire de nos avis : 

In [229]:
vect_count.get_feature_names_out()[:50] # 50 premiers mots ("types" du vocabulaire)

array(['00', '00p', '0p', '0petite', '0r', '0verall', '10', '100', '1000',
       '100lbs', '101', '102', '102lbs', '103', '103lbs', '104', '105',
       '105lbs', '106', '106lbs', '107', '107lb', '107lbs', '108', '109',
       '109lbs', '10p', '10s', '10th', '11', '110', '110lb', '110lbs',
       '111lbs', '112', '112lbs', '112llbs', '113', '113lbs', '114',
       '114lb', '114lbs', '115', '115ish', '115lbs', '115llbs', '116',
       '116bs', '116lb', '116lbs'], dtype=object)

In [230]:
vect_count.get_feature_names_out()[-50:] # 50 derniers mots ("types" du vocabulaire)

array(['yepeee', 'yes', 'yesterday', 'yesteryear', 'yet', 'yey', 'yield',
       'yikes', 'yippee', 'yo', 'yoga', 'yogis', 'yoke', 'york', 'you',
       'young', 'younger', 'your', 'youre', 'yourself', 'youth',
       'youthful', 'yr', 'yrs', 'yuck', 'yucky', 'yummiest', 'yummy',
       'yup', 'zag', 'zara', 'zero', 'zig', 'zip', 'zipepr', 'ziploc',
       'zipped', 'zipper', 'zippered', 'zippers', 'zippie', 'zipping',
       'zips', 'zombie', 'zone', 'zoolander', 'zoom', 'zooming', 'zuma',
       'ã¼ber'], dtype=object)

Taille de notre vocabulaire :

In [231]:
len(vect_count.get_feature_names_out()) 

9587

#### Création matrice document-termes

Nous allons créer la matrice document-termes avec le même vectoriseur.

In [232]:
X_train_vectorized_count = vect_count.transform(X_train)
X_train_vectorized_count

<8847x9587 sparse matrix of type '<class 'numpy.int64'>'
	with 386829 stored elements in Compressed Sparse Row format>

In [233]:
X_valid_vectorized_count = vect_count.transform(X_valid)
X_test_vectorized_count = vect_count.transform(X_test)

A présent, nous allons prendre en compte les bi-grammes dans notre vocabulaire. 

In [234]:
vect_count_bigrams = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)
X_train_vectorized_count_bigrams = vect_count_bigrams.transform(X_train)
X_valid_vectorized_count_bigrams = vect_count_bigrams.transform(X_valid)
X_test_vectorized_count_bigrams = vect_count_bigrams.transform(X_test)

In [235]:
len(vect_count_bigrams.get_feature_names_out())

17321

Nous avons presque 2 fois plus de vocabulaire avec inclusion des bigrammes.

###  Numérique continu : TF-IDF (ou autres pondérations)

Nous allons limiter le vocabulaire à des termes qui apparaissent au moins 5 fois dans le document.

In [236]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [237]:
vect_tfidf = TfidfVectorizer(min_df=5).fit(X_train)

In [238]:
len(vect_count.get_feature_names_out()), len(vect_tfidf.get_feature_names_out())

(9587, 3232)

La réduction de la taille du vocadulaire est importante et est due au paramètre min_df=5 : on a quasiment 3 fois moins de termes !

Nous allons vectoriser les jeux de données. 

In [239]:
# Vectorisation des corpus d'entrainement, de validation et de test
X_train_vectorized_tfidf = vect_tfidf.transform(X_train)
X_valid_vectorized_tfidf = vect_tfidf.transform(X_valid)
X_test_vectorized_tfidf = vect_tfidf.transform(X_test)

## Classification des textes

Nous allons réaliser une classification en utilisant plusieurs modèles afin de comparer les performances. 

In [240]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

### Modèles de référence faibles

#### Choix aléatoire

Nous allons d'abord choisir un modèle où toutes les classes ont la même probabilité d'être choisies ou bien le prédicteur respecte la disctribution des classes dans les données d'entrainement.

In [241]:
from sklearn.dummy import DummyClassifier

Prédiction proportionnelle à la distribution des classes dans les données d'entraînement :

In [242]:
random_prop_class = DummyClassifier(strategy='stratified').fit(X_train_vectorized_tfidf,
                                                               y_train)
predictions_valid = random_prop_class.predict(X_valid_vectorized_tfidf)
conf_mat = confusion_matrix(y_valid, predictions_valid)

In [243]:
print(conf_mat)

[[ 29 120 187]
 [110 337 546]
 [164 569 888]]


Prédiction uniforme : 

In [244]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_vectorized_tfidf,
                                                         y_train)
predictions_valid = random_uniform.predict(X_valid_vectorized_tfidf)
predictions_valid

array([ 0,  1,  1, ...,  1,  0, -1], dtype=int64)

In [245]:
conf_mat = confusion_matrix(y_valid, predictions_valid)

In [246]:
print(conf_mat)

[[106 133  97]
 [340 320 333]
 [526 562 533]]


In [247]:
accuracy_score(y_valid, predictions_valid)

0.32508474576271185

In [248]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.11      0.32      0.16       336
           0       0.32      0.32      0.32       993
           1       0.55      0.33      0.41      1621

    accuracy                           0.33      2950
   macro avg       0.33      0.32      0.30      2950
weighted avg       0.42      0.33      0.35      2950



#### Prédiction constante de la classe majoritaire

Nous allons d'abord identifier la répartition des classes dans les données d'entrainement.

In [254]:
class_distribution

,num_examples
class,
1,6497
0,4075
-1,1225


In [250]:
maj = DummyClassifier(strategy='most_frequent').fit(X_train_vectorized_tfidf, y_train)
predictions_valid = maj.predict(X_valid_vectorized_tfidf)
predictions_valid

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [258]:
import numpy as np 

maj_class = (class_distribution.index[class_distribution.perc_examples ==
                                      np.amax(class_distribution.perc_examples)][0])
maj_class

1

In [259]:
np.all(predictions_valid == maj_class)

True

In [260]:
maj.score(X_valid_vectorized_tfidf, y_valid)

0.5494915254237288

In [261]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       336
           0       0.00      0.00      0.00       993
           1       0.55      1.00      0.71      1621

    accuracy                           0.55      2950
   macro avg       0.18      0.33      0.24      2950
weighted avg       0.30      0.55      0.39      2950



c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Classifieur naïf bayésien

In [262]:
from sklearn.naive_bayes import MultinomialNB

In [263]:
model_nb = MultinomialNB().fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_valid_vectorized_tfidf)

In [264]:
accuracy_score(y_valid, predictions_valid)

0.6823728813559322

In [265]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.62      0.01      0.03       336
           0       0.57      0.50      0.54       993
           1       0.73      0.93      0.82      1621

    accuracy                           0.68      2950
   macro avg       0.64      0.48      0.46      2950
weighted avg       0.66      0.68      0.63      2950



### Régression logistique

In [266]:
from sklearn.linear_model import LogisticRegression

In [267]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                              max_iter=200).fit(X_train_vectorized_count, y_train)

c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [268]:
predictions_valid = model_lr.predict(X_valid_vectorized_count)

In [269]:
accuracy_score(y_valid, predictions_valid)

0.6752542372881356

In [270]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.53      0.42      0.47       336
           0       0.55      0.58      0.56       993
           1       0.78      0.79      0.78      1621

    accuracy                           0.68      2950
   macro avg       0.62      0.59      0.60      2950
weighted avg       0.67      0.68      0.67      2950



In [271]:
def print_n_strongly_associated_features(vectoriser, model, n):
    feature_names = np.array(vectoriser.get_feature_names_out())

    for i in range(3):
        class_name = model.classes_[i]
        print("CLASSE {}".format(class_name))
        idx_coefs_sorted = model.coef_[i].argsort() # ordre croissant
        print("Les dix variables ayant l'association négative la plus forte " + 
              "avec la classe {} :\n{}\n".format(class_name,
                                                 feature_names[idx_coefs_sorted[:n]]))
        idx_coefs_sorted = idx_coefs_sorted[::-1] # ordre décroissant
        print("Les dix variables ayant l'association positive la plus forte " +
              "avec la classe {} :\n{}\n"
              .format(class_name,
                      feature_names[idx_coefs_sorted[:n]]))
        print()

Examinons les variables (termes) ayant l'association la plus forte avec chaque classe.

In [272]:
print_n_strongly_associated_features(vect_count, model_lr, 10)

CLASSE -1
Les dix variables ayant l'association négative la plus forte avec la classe -1 :
['nude' 'wait' 'compliments' 'easy' 'little' '34' 'amazing' 'fun' 'hoped'
 'complaint']

Les dix variables ayant l'association positive la plus forte avec la classe -1 :
['flow' 'poor' 'weird' 'instructions' 'awful' 'unflattering' 'bummed'
 'money' 'returning' 'disappointing']


CLASSE 0
Les dix variables ayant l'association négative la plus forte avec la classe 0 :
['machine' 'bonus' 'risk' '140' 'heat' 'sleeved' 'fixed' 'door' 'string'
 'street']

Les dix variables ayant l'association positive la plus forte avec la classe 0 :
['cap' 'yarn' 'clearance' 'sparkle' 'stitches' 'sundry' 'snatched' 'sleek'
 'opposed' 'pulls']


CLASSE 1
Les dix variables ayant l'association négative la plus forte avec la classe 1 :
['disappointing' 'cap' 'idea' 'poor' 'seemed' 'bummed' 'returning'
 'disappointed' 'taste' 'strange']

Les dix variables ayant l'association positive la plus forte avec la classe 1 :
['mach

COMMENTAIRE A METTRE

In [273]:
model_lr = LogisticRegression(multi_class='multinomial',
                              solver='lbfgs').fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_tfidf)

c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [274]:
accuracy_score(y_valid, predictions_valid)

0.703728813559322

In [275]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.67      0.28      0.39       336
           0       0.59      0.60      0.59       993
           1       0.77      0.85      0.81      1621

    accuracy                           0.70      2950
   macro avg       0.68      0.58      0.60      2950
weighted avg       0.70      0.70      0.69      2950



In [276]:
feature_names = np.array(vect_tfidf.get_feature_names_out())
idx_tfidf_sorted = X_train_vectorized_tfidf.max(0).toarray()[0].argsort()
print("TF-IDF le moins élevé : {}".format(feature_names[idx_tfidf_sorted[:10]]))
print("TF-IDF le plus élevé : {}".format(feature_names[idx_tfidf_sorted[:-11:-1]]))

TF-IDF le moins élevé : ['pros' 'shouldn' 'tighten' 'secondly' 'framed' 'wo' 'xspetite' '30d'
 'dingy' 'fo']
TF-IDF le plus élevé : ['amp' 'sweet' 'loved' 'simple' 'she' 'cute' 'fits' 'hei' 'decent' 'rough']


Nous faisons avec les mêmes paramètres mais avec le vectoriseur à unigrammes et bigrammes.

In [277]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=500).fit(X_train_vectorized_count_bigrams, y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_count_bigrams)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

In [ ]:
print_n_strongly_associated_features(vect_count_bigrams, model_lr, 10)

### SVM

In [278]:
from sklearn.svm import SVC

In [279]:
model_svm = SVC(kernel='linear', C=0.1).fit(X_train_vectorized_count_bigrams, y_train)
predictions_valid = model_svm.predict(X_valid_vectorized_count_bigrams)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))